In [1]:
from ASV_dl_func import *

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
from omegaconf import OmegaConf

config = OmegaConf.load("config.yaml")

la_metadata = config.datasets.LA.metadata
la_flac_folders = config.datasets.LA.flac
la_columns = config.datasets.LA.columns

print("LA metadata:", la_metadata)
print("LA flac folders:", la_flac_folders)
print("LA columns:", la_columns)

In [ ]:
# final_df = prepare_dataframe(all_data, sample_size=5000)
# final_df.to_csv("prepared_audio_data2.csv")
# final_df = pd.read_csv("prepared_audio_data2.csv")

In [ ]:
X_train = pd.read_csv("csvki/asv2019_inthewild_dataset.csv")

In [ ]:
feature_extractors_map = {
    'cqcc': extract_cqcc,
    'gtcc': extract_gtcc, 
    'mel-spect': extract_mel_spectrogram,
    'mfcc': extract_mfcc,
    'lfcc': extract_lfcc,
    'wpt': extract_wpt,
    # 'Formants': analyze_formants_and_silence
}

In [ ]:
X_train_1 = extract_features(X_train, feature_extractors_map) 
X_train_1 = X_train_1.dropna()

In [ ]:
X_test = pd.read_csv("csvki/test_asv2016_data.csv")
extract_features(X_test, feature_extractors_map) 
X_test = X_test.dropna()

# Sprawdzenie zbalansowanai

In [ ]:
X_train_bez_scalowania = balance_func(X_train_1, col_name='label')
X_test_bez_scalowania = balance_func(X_test, col_name='label')

# Cechy

In [ ]:
feature_cols = ['cqcc', 'mel-spect', 'mfcc', 'wpt']
trained_models = train_all_features(X_train_bez_scalowania, feature_cols, epochs=100, label_col="label", test_df=X_test_bez_scalowania)

# BiLSTM

In [ ]:
X_train_bilstm = filtr_nan(X_train_bez_scalowania)
X_train_bilstm['cqcc'] = X_train_bilstm['cqcc'].apply(transpose_cqcc)

initial_len = len(X_train_bilstm)
final_df = X_train_1[X_train_bilstm['cqcc'].notnull()]

final_df_balanced = balance_func(final_df, col_name='label')

train_df, test_df, scaler = prepare_train_test_data(final_df_balanced, label_name='label')

model, test_loader = BiLSTM_model(train_df, test_df, num_epochs=100)

In [ ]:
eval_model(trained_models['cqcc'][0], train_df, test_df, trained_models['cqcc'][1])

In [ ]:
train_df['cqcc']